In [60]:
import os
import numpy as np
from PIL import Image
import pandas as pd
from scipy.spatial import distance
from skimage.feature import hog
from sklearn.neighbors import KNeighborsClassifier


def load_dataset(image_path):
    dataset = {
        'label' : [],
        'image' : []
    }

    for dir in os.listdir(image_path):
        if dir not in '.DS_Store':
                #print(dir)
                class_path = os.path.join(image_path, dir)
                for image in os.listdir(class_path):
                    dataset['image'].append(np.array(Image.open(os.path.join(image_path, dir , image))))
                    dataset['label'].append(int(dir))
                    
    return pd.DataFrame(dataset)

val_set = load_dataset('files/MNIST/MNIST/val')
train_set = load_dataset('files/MNIST/MNIST/train')

In [61]:
val_set['image'] = val_set['image'].apply(lambda img : hog(img, orientations=8, feature_vector=True))
train_set['image'] = train_set['image'].apply(lambda img : hog(img, orientations=8, feature_vector=True))

In [62]:

def build_model(x):
    model = {}
    for i in range(10):
    #    model[i] = x[x['label']==i]['image'].apply(lambda x: x.sum(axis=0)).mean(axis=0) # calculate mean vector of all image in class
        model[i] = x[x['label']==i]['image'].mean()
    return model

def classify(x, model):
    dists = np.zeros(10)
    
    for key in model:
        dists[key] = distance.cosine(x, model[key])

    return dists.argmin()


model = build_model(train_set)
#classify(val_set['image'].iloc[1020].sum(axis=0), model)
# predictions = val_set.apply(lambda x : classify(x['image'].sum(axis=0), model),axis =1 )
predictions = val_set.apply(lambda x : classify(x['image'], model),axis =1 )

mm = pd.concat([val_set['label'], predictions], axis=1)
mm.apply(lambda x : 1 if x['label']==x[0] else 0, axis=1).sum()/(len(val_set))

0.8518333333333333

In [55]:

#sklearn KNN
knn= KNeighborsClassifier(n_neighbors=3)
knn.fit(np.vstack(train_set['image']),train_set['label'])
knn.score(np.vstack(val_set['image']), val_set['label']) 

0.8255833333333333

In [58]:
val_set['image'].iloc[0].shape

(18,)